In [2]:
from pygraylog.graylogapi import GraylogAPI
import json
import sys
import numpy as np
import pandas as pd

In [3]:
log_level = 0

In [37]:
graylog_con={
        "user":"merger",
        "password":"",
        "url":"http://graylog-ui.roamability.com:9000/api"
    }

In [4]:
graylog_con={
        "user":"pavel",
        "password":"O777ver7k1",
        "url":"http://graylog-ui.roamability.com:9000/api"
    }

In [5]:
api = GraylogAPI(
    graylog_con["url"],
    graylog_con["user"], 
    graylog_con["password"])

In [27]:
import datetime
#format like 2017-03-06T16:28
dateFrom = datetime.datetime.strptime('2018-01-12T05:00', '%Y-%m-%dT%H:%M')
dateTo = datetime.datetime.strptime('2018-01-12T12:00', '%Y-%m-%dT%H:%M')
#query='protocol:ss7 AND opCode_localValue:2 AND TCAP_type:Begin AND e212_imsi:425191201068102 AND tcap_otid:"a2:2a:0c:40"'
query='protocol:ss7 AND (tcap_otid:"a2:2a:0c:40" OR tcap_dtid:"a2:2a:0c:40")'

In [30]:
gres=api.search.universal.absolute.get(query=query, from_=dateFrom, to=dateTo)

In [31]:
gres

u'{"query":"protocol:ss7 AND (tcap_otid:\\"a2:2a:0c:40\\" OR tcap_dtid:\\"a2:2a:0c:40\\")","built_query":"{\\"from\\":0,\\"size\\":150,\\"query\\":{\\"bool\\":{\\"must\\":{\\"query_string\\":{\\"query\\":\\"protocol:ss7 AND (tcap_otid:\\\\\\"a2:2a:0c:40\\\\\\" OR tcap_dtid:\\\\\\"a2:2a:0c:40\\\\\\")\\",\\"allow_leading_wildcard\\":false}},\\"filter\\":{\\"bool\\":{\\"must\\":{\\"range\\":{\\"timestamp\\":{\\"from\\":\\"2018-01-12 04:00:00.000\\",\\"to\\":\\"2018-01-12 11:00:00.000\\",\\"include_lower\\":true,\\"include_upper\\":true}}}}}}},\\"sort\\":[{\\"timestamp\\":{\\"order\\":\\"desc\\"}}]}","used_indices":[{"index_name":"graylog_45","begin":"1970-01-01T00:00:00.000Z","end":"1970-01-01T00:00:00.000Z","calculated_at":"2018-02-02T03:41:53.726Z","took_ms":0},{"index_name":"graylog_36","begin":"2018-01-11T20:58:35.461Z","end":"2018-01-14T00:41:28.217Z","calculated_at":"2018-01-14T00:43:34.379Z","took_ms":4767}],"messages":[{"highlight_ranges":null,"message":{"sccp_called_nai":"4","m3u

In [32]:
def outLog(message, level=1):
    if(level <= log_level):
        print (message)
        sys.stdout.flush()

In [33]:
def LoadGraylogResult(exportsearch_results):
        jsonResult = json.loads(exportsearch_results)
        outputResult = list()
        i=0
        if "total_results" in jsonResult:
            outLog("Graylog search found {} messages".format(jsonResult["total_results"]),0)

        # Get the results and store them
            for result in jsonResult["messages"]:  
                if "message" in  result and isinstance(result["message"], dict):
                    outputResult.append(result["message"])
                    i+=1
                    if i%100==0: outLog("loaded {} records from Graylog".format(i),0)
            outLog("Graylog search load {} messages".format(len(outputResult)),0)
        else:
            outLog("Graylog search error: {}".format(exportsearch_results),0)

        return outputResult

In [34]:
pgres=LoadGraylogResult(gres)

Graylog search found 59 messages
Graylog search load 59 messages


In [35]:
pgres

[{u'TCAP_type': u'End',
  u'_id': u'96eac9ef-f785-11e7-8ae9-00505698f35d',
  u'collector_node_id': u'graylog-collector-sidecar',
  u'facility': u'filebeat',
  u'file': u'/mnt/tracing1ch/sigtran-public/sigtran-public_07105_20180112112713.pcap',
  u'frame_number': u'20803',
  u'frame_time_epoch': u'1515753027.387814327',
  u'gl2_remote_ip': u'172.19.11.192',
  u'gl2_remote_port': 46264,
  u'gl2_source_collector': u'b3d2019c-476e-439e-8764-043b24bcda9d',
  u'gl2_source_input': u'5a12819aab324f51d795a633',
  u'gl2_source_node': u'be115c61-9b16-42d7-810c-8f0e4af6a69c',
  u'input_type': u'log',
  u'm3ua_DPC': u'800',
  u'm3ua_OPC': u'781',
  u'message': u'12/01/2018 11:30:27.387814 \tprotocol=ss7\tfile=/mnt/tracing1ch/sigtran-public/sigtran-public_07105_20180112112713.pcap\tframe_time_epoch=1515753027.387814327\tprotocol_version=2\tsccp_called_ri=0\ttcap_dtid=a2:2a:0c:40\tm3ua_OPC=781\tprotocol_name=map\tTCAP_type=End\tsccp_calling_ri=0\tsccp_calling_ssn=6\tframe_number=20803\tsccp_calling_d

In [36]:
data=pgres

In [37]:
data[0]['m3ua_OPC']

u'781'

In [38]:
columns=list(data[0].keys())
values = list(data[0].values())
arr_len = len(values)
pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)

,protocol,gl2_source_collector,sccp_calling_ssn,tcap_dtid,opCode_localValue,sccp_message_type,m3ua_DPC,file,message,sccp_called_nai,...,offset,sccp_called_ri,gl2_source_node,m3ua_OPC,sccp_called_digits,TCAP_type,streams,sccp_calling_ri,_id,frame_number
0,ss7,b3d2019c-476e-439e-8764-043b24bcda9d,6,a2:2a:0c:40,56,0x00000009,800,/mnt/tracing1ch/sigtran-public/sigtran-public_...,12/01/2018 11:30:27.387814 \tprotocol=ss7\tfil...,4,...,6172175,0,be115c61-9b16-42d7-810c-8f0e4af6a69c,781,919434399505,End,[000000000000000000000001],0,96eac9ef-f785-11e7-8ae9-00505698f35d,20803


In [41]:
columns=list(data[0].keys())
values = list(data[0].values())
arr_len = len(values)
df=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
for i in range(len(data)-1):
    columns=list(data[i+1].keys())
    values = list(data[i+1].values())
    arr_len = len(values)
    df1=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
    df=pd.concat([df, df1],axis=0,join='outer')
df=df.reset_index(drop=True)
df.head(3)

,TCAP_type,_id,collector_node_id,e164_msisdn,e212_imsi,e212_mcc,e212_mnc,facility,file,frame_number,...,sccp_class,sccp_message_type,source,streams,tags,tcap_application_context_name,tcap_dtid,tcap_otid,timestamp,type
0,End,96eac9ef-f785-11e7-8ae9-00505698f35d,graylog-collector-sidecar,NaN,NaN,NaN,NaN,filebeat,/mnt/tracing1ch/sigtran-public/sigtran-public_...,20803,...,0,0x00000009,admin,[000000000000000000000001],[tracer],0.4.0.0.1.0.14.2,a2:2a:0c:40,NaN,2018-01-12T10:30:27.387Z,pcap
0,Begin,b890ae26-f785-11e7-8ae9-00505698f35d,graylog-collector-sidecar,NaN,425019612539521,425,1,filebeat,/mnt/tracing2ch/sigtran-public/sigtran-public_...,9943,...,1,0x00000009,admin,[000000000000000000000001],[tracer],0.4.0.0.1.0.14.2,NaN,a2:2a:0c:40,2018-01-12T10:30:27.375Z,pcap
0,Begin,b85a82f2-f785-11e7-8ae9-00505698f35d,graylog-collector-sidecar,46731726312,240400000038251,240,40,filebeat,/mnt/tracing2ch/sigtran-public/sigtran-public_...,7046,...,NaN,0x0000000a,admin,[000000000000000000000001],[tracer],0.4.0.0.1.0.25.3,NaN,a2:2a:0c:40,2018-01-12T10:29:55.578Z,pcap
0,Begin,b85a82f1-f785-11e7-8ae9-00505698f35d,graylog-collector-sidecar,46731726312,240400000038251,240,40,filebeat,/mnt/tracing2ch/sigtran-public/sigtran-public_...,7045,...,1,0x00000009,admin,[000000000000000000000001],[tracer],0.4.0.0.1.0.25.3,NaN,a2:2a:0c:40,2018-01-12T10:29:55.548Z,pcap
0,End,6581bd22-f782-11e7-8ae9-00505698f35d,graylog-collector-sidecar,NaN,NaN,NaN,NaN,filebeat,/mnt/tracing2ch/sigtran-public/sigtran-public_...,16630,...,0,0x00000009,admin,[000000000000000000000001],[tracer],0.4.0.0.1.0.14.3,a2:2a:0c:40,NaN,2018-01-12T10:01:33.924Z,pcap


In [40]:
df[[
    'm3ua_OPC','m3ua_DPC',
    'sccp_calling_nai','sccp_calling_digits','sccp_calling_ri','sccp_calling_ssn',
    'sccp_called_nai','sccp_called_digits','sccp_called_ri','sccp_called_ssn',
    'sccp_class','sccp_message_type','protocol_name','protocol','protocol_version','opCode_localValue','TCAP_type',
    'tcap_otid','tcap_dtid',
    'e164_msisdn','e212_imsi']].head()

,m3ua_OPC,m3ua_DPC,sccp_calling_nai,sccp_calling_digits,sccp_calling_ri,sccp_calling_ssn,sccp_called_nai,sccp_called_digits,sccp_called_ri,sccp_called_ssn,...,sccp_message_type,protocol_name,protocol,protocol_version,opCode_localValue,TCAP_type,tcap_otid,tcap_dtid,e164_msisdn,e212_imsi
0,781,800,4,97254120634,0,6,4,919434399505,0,7,...,0x00000009,map,ss7,2,56,End,NaN,a2:2a:0c:40,NaN,NaN
0,801,782,4,919434399505,0,7,4,972549612539521,0,6,...,0x00000009,map,ss7,NaN,56,Begin,a2:2a:0c:40,NaN,NaN,425019612539521
0,1962,2506,4,46731726312,0,8,4,46731726312,0,8,...,0x0000000a,SCCP,ss7,NaN,44,Begin,a2:2a:0c:40,NaN,46731726312,240400000038251
0,2506,1962,4,46731726312,0,8,4,46731726312,0,8,...,0x00000009,GSM,ss7,NaN,44,Begin,a2:2a:0c:40,NaN,46731726312,240400000038251
0,7138,701,4,972559900050,0,6,4,972559900040,0,7,...,0x00000009,map,ss7,NaN,56,End,NaN,a2:2a:0c:40,NaN,NaN


In [ ]:
#Сборка датафрейма XDR

columns=list(df.keys())
df_xdr=pd.DataFrame(columns=columns)
for i in range(len(df)):
    if df.iloc[i]['TCAP_type']=='Begin' and df.iloc[i]['e212_imsi']=='425019612539521':
        values = list(df.iloc[i].values)
        arr_len = len(values)
        df_temp=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
        df_xdr=pd.concat([df_xdr, df_temp],axis=0,join='outer')
        #df_temp.drop([i],inplace=True)
df_xdr=df_xdr.reset_index(drop=True)
df_xdr